### Problem Statement

The assignment for Week 8 is to use Cython to speed up the code you wrote for assignment 2 (SPICE simulation). Note that the grade is not based on how much you speed up, but on how well you are able to analyze the code and explain your optimizations. Your documentation here is particularly important. Even if you try techniques but don't see a speedup, you should try to explain why it didn't work.

## Cython :

Cython is a programming language that makes writing C extensions for the Python language as easy as Python itself. It aims to become a superset of the Python language which gives it high-level, object-oriented, functional, and dynamic programming. Its main feature on top of these is support for optional static type declarations as part of the language. The source code gets translated into optimized C/C++ code and compiled as Python extension modules. This allows for both very fast program execution and tight integration with external C libraries, while keeping up the high programmer productivity for which the Python language is well known.

In [4]:
import cmath # for complex numbers
pi=cmath.pi

- cmath library is used for dealing with complex numbers.

In [7]:
%load_ext Cython 
# To load the Cython extension from within the Jupyter notebook

In [8]:
%%cython --annotate
import cython
import cmath # for complex numbers
pi=cmath.pi
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef gauss(list A,list b):
    cdef int count
    cdef int i
    cdef int j
    cdef int k
    cdef int l
    cdef int p
    cdef complex norm
    try:
        if type(A)!=list:
            A=A.tolist()
        if type(b)!=list:b=b.tolist()# making sure that the inputs are lists if not converting them to lists.
    except: return "Error! A has to be a square matrix and b has to be a column matrix and len(A) should be equal to len(b)"
    if len(A)!=len(b): return "Error! no.of elements in A has to be equal to no. of elements in b"
    cdef list arg_mat=(A.copy())
    #creatiing argumented matrix of A and b i.e., arg_mat=[A|b]
    for i in range(len(arg_mat)):
        arg_mat[i].append(b[i])  
    i=0 # step 1
    while i<len(A):
        if arg_mat[i][i]!=0 :
            n=arg_mat[i][i]
            for j in range(len(A)+1): #making the the [i][i]th element as 1 if it is non zero.
                arg_mat[i][j]/=n
        else :
            count=0
            #step 4
            for k in range(i+1,len(A)):
                if arg_mat[k][i]!=0:
                    arg_mat[k],arg_mat[i]=arg_mat[i],arg_mat[k] 
                    count+=1
                    break
            #step 5
            if count==0: 
                arg_mat[len(arg_mat)-1],arg_mat[i]=arg_mat[i],arg_mat[len(arg_mat)-1] 
                i+=1
                continue
            #step 2
            for j in range(len(A)+1):
                n=arg_mat[i][i]
                arg_mat[i][j]=(arg_mat[i][j]/n)
            #step 3 
        for p in range(len(A)):
            if p!=i:
                norm=arg_mat[p][i]
                for l in range(len(A)+1):
                    arg_mat[p][l]-=(norm)*arg_mat[i][l]
        i+=1 #step 6
   # print("The row reduced echlon form of the given system of equations is ")
   # print(arg_mat)
    cdef list sol=[0 for i in range(len(A))]
    for i in range(len(A)):
        A_part=arg_mat[i][:len(arg_mat)]
        b_part=arg_mat[i][len(arg_mat)]
        if A_part==[0 for i in range(len(A))] and b_part==0: # step 8
            return "infinitely many solutions"
        elif A_part==[0 for i in range(len(A))] and b_part!=0: #step 9
            return "no solution"
    #step 7
    for i in range(len(A)):
        for j in range(len(A)):
            if arg_mat[i][j]==1:
                sol[j]=arg_mat[i][len(arg_mat)]
    # print("the solution is")
    return sol
cdef to_dec(str s):
    cdef int e_pos
    cdef int j
    e_pos=0
    for j in range(len(s)):
        if s[j]=='e':
            e_pos=j
            break
    return int(s[:j])*pow(10,int(s[j+1:]))

cpdef ckt(str filename):
    cdef int i,j
    file=open(filename,'r') #step 1: to read file
    cdef list lis=[]
    for each in file:     #step 2: to store the contents of file into list
        lis.append(each.split())
    file.close()
    cdef int start=0
    cdef int end=0
    cdef int com
    freq=0
# to remove junk , .circuit and .end i.e., taking only components of the circuit
    for i in range(len(lis)): #step 3
        if lis[i]==['.circuit']:
            start=i
        if lis[i]==[".end"]:
            end=i
        if lis[i]!=[]: 
            if lis[i][0]==".ac": freq=2*pi*int(lis[i][2]) #converting the given frequency to angular frequency
    lis=lis[start+1:end] #step 4
#step 5 : to remove comments
    for i in range(len(lis)):
        com=0
        for j in range(len(lis[i])):
            if lis[i][j][0]=="#":
                com=j
        if com!=0:lis[i]=lis[i][:com]
    #step 6: finding all the nodes and total no. of voltage sources
    cdef list nodes=[]
    cdef int nv=0
    for i in range(len(lis)):
        if lis[i][0][0]=="V": nv+=1
        if lis[i][1]=='GND': lis[i][1]='0'
        if lis[i][2]=='GND': lis[i][2]='0'
        if lis[i][1] not in nodes:
            nodes.append(lis[i][1])
        if lis[i][2] not in nodes:
            nodes.append(lis[i][2])
    nodes.remove('0')
    #step 7 : creating coefficient and constant matrices
    cdef list A=[[complex(0) for i in range(len(nodes)+nv)] for j in range(len(nodes)+nv)]
    cdef list b=[0 for j in range(len(nodes)+nv)]
    cdef int k=0
    #step 8 : analysing elements in the matrices and making appropriate changes in the matrices
    for j in range(len(lis)):
        n=lis[j][1]
        m=lis[j][2]
        if n.isdigit() :n=int(lis[j][1])
        else : n=int(lis[j][1][1:]) #line 1
        if m.isdigit() :m=int(lis[j][2])
        else : m=int(lis[j][2][1:])
        if lis[j][0][0]=='R': # for resistor
            if n!=0 and m!=0:
                n-=1
                m-=1
                if lis[j][3].isdigit():
                    A[n][n]+=(1/int(lis[j][3]))
                    if n!=m:
                        A[n][m]-=(1/int(lis[j][3]))
                        A[m][n]-=(1/int(lis[j][3]))
                        A[m][m]+=(1/int(lis[j][3]))
                else:
                    A[n][n]+=(1/to_dec(lis[j][3]))
                    if n!=m:
                        A[n][m]-=(1/to_dec(lis[j][3]))
                        A[m][n]-=(1/to_dec(lis[j][3]))
                        A[m][m]+=(1/to_dec(lis[j][3]))
            elif n==0:
                if lis[j][3].isdigit(): A[m-1][m-1]+=(1/int(lis[j][3]))
                else : A[m-1][m-1]+=(1/to_dec(lis[j][3]))
            elif m==0:
                if lis[j][3].isdigit(): A[n-1][n-1]+=(1/int(lis[j][3]))
                else : A[n-1][n-1]+=(1/to_dec(lis[j][3]))
        elif lis[j][0][0]=='L': # for inductors
            if n!=0 and m!=0:
                n-=1
                m-=1
                if lis[j][3].isdigit():
                    A[n][n]+=(1/(int(lis[j][3])*freq*(1j)))
                    if n!=m:
                        A[n][m]-=(1/(int(lis[j][3])*freq*(1j)))
                        A[m][n]-=(1/(int(lis[j][3])*freq*(1j)))
                        A[m][m]+=(1/(int(lis[j][3])*freq*(1j)))
                else:
                    A[n][n]+=(1/to_dec(lis[j][3]))
                    if n!=m:
                        A[n][m]-=(1/(to_dec(lis[j][3])*freq*(1j)))
                        A[m][n]-=(1/(to_dec(lis[j][3])*freq*(1j)))
                        A[m][m]+=(1/(to_dec(lis[j][3])*freq*(1j)))
            elif n==0:
                if lis[j][3].isdigit(): A[m-1][m-1]+=(1/(int(lis[j][3])*freq*(1j)))
                else : A[m-1][m-1]+=(1/(to_dec(lis[j][3])*freq*(1j)))
            elif m==0:
                if lis[j][3].isdigit(): A[n-1][n-1]+=(1/(int(lis[j][3])*freq*(1j)))
                else : A[n-1][n-1]+=(1/(to_dec(lis[j][3])*freq*(1j)))
        elif lis[j][0][0]=='C': # for capacitors
            if n!=0 and m!=0:
                n-=1
                m-=1
                if lis[j][3].isdigit():
                    A[n][n]+=((int(lis[j][3])*freq*(1j)))
                    if n!=m:
                        A[n][m]-=((int(lis[j][3])*freq*(1j)))
                        A[m][n]-=((int(lis[j][3])*freq*(1j)))
                        A[m][m]+=((int(lis[j][3])*freq*(1j)))
                else:
                    A[n][n]+=(to_dec(lis[j][3]))
                    if n!=m:
                        A[n][m]-=(to_dec(lis[j][3])*freq*(1j))
                        A[m][n]-=(to_dec(lis[j][3])*freq*(1j))
                        A[m][m]+=(to_dec(lis[j][3])*freq*(1j))
            elif n==0:
                if lis[j][3].isdigit(): A[m-1][m-1]+=cmath.rect(freq*int(lis[j][3]),pi/2)
                else : A[m-1][m-1]+=((to_dec(lis[j][3])*freq*(1j)))
            elif m==0:
                if lis[j][3].isdigit(): A[n-1][n-1]+=((int(lis[j][3])*freq*(1j)))
                else : A[n-1][n-1]+=((to_dec(lis[j][3])*freq*(1j)))
        elif lis[j][0][0]=='V': #for voltage sources n1(n) is positive n2(m) is negative
            if m!=0:           #current through voltage source is directed from n1(n) to n2(m)
                m-=1
                A[len(nodes)+k][m]-=1
                A[m][len(nodes)+k]-=1
            if n!=0:
                n-=1
                A[len(nodes)+k][n]+=1
                A[n][len(nodes)+k]+=1
            if lis[j][3]=="dc" : b[len(nodes)+k]+=int(lis[j][4])
            elif lis[j][3]=="ac" :b[len(nodes)+k]+=cmath.rect(int(lis[j][4]),int(lis[j][5]))
            k+=1
        elif lis[j][0][0]=='I': # for current sources flowing from n2 to n1 i.e.s flowing into n1 and out of n2
            if m!=0:
                m-=1
                if lis[j][3]=="dc" :b[m]-=int(lis[j][4])
                elif lis[j][3]=="ac" : b[m]-=cmath.rect(int(lis[j][4]),int(lis[j][5]))
            if n!=0:
                n-=1
                if lis[j][3]=="dc" :b[n]+=int(lis[j][4])
                elif lis[j][3]=="ac" :b[n]+=cmath.rect(int(lis[j][4]),int(lis[j][5]))
    return gauss(A,b)      #step 9 : To find the solution of the matrices   

- The code used in assignment-2 is optimized using Cython
- We can define a Cython cell by writing %%cython on top of it.
##### Note that each cell will be compiled into a seperate extension module.So If you use a package in a Cython cell, you will have to import this package in the same cell. It's not enough to have imported the package in a previous cell. Cython will tell you that there are 'undefined global names' at compilation time if u don't comply.
- %%cython --annotate produce a colorized HTML version of the source.Yellow lines hint at Python interaction.
- Lines are colored according to the typedness - whitelines will translate to pure C, while lines that require the Python C-API are yellow (darker as they translate to more C-API interaction).
- Lines that translate to C code have a plus (+) in front and can be clicked to show the generated code.
- Python function calls can be expensive – in Cython doubly so because one might need to convert to and from Python objects to do the call. 
- Cython provides a way for declaring a C-style function, the Cython specific cdef statement, as well as the @cfunc decorator to declare C-style functions in Python syntax. Both approaches are equivalent and produce the same C code.
- A side effect of cdef (and @cfunc decorator) is that the function no longer visible from Python space,as Python wouldn't know how to call it.
- Using the cpdef keyword instead of cdef, a Pyhton wrapper is also created, so that the function is available both from Cython(fast,passing typed values directly).
- In fact cpdef does not just provide a Pyhton wrapper, it also installs logic to allow the method to be overridden by python methods, even when called from within cython.
- This does a tiny overhead compared to cdef methods.
- Cython provides a @ccall decorator which provides the same functionality as cpdef keyword.
- Adding static types of few variables can make a much large difference in performance of a program.
- Declaration of types of variables can be done using cdef function.
- We can type caste the iterator's variable, So that the for loop will be compiled to pure C code.
- And I have used a few compiler directives:
1. @cython.cdivision(True/False) : 
   If set to False, Cython will adjust the remainder and quotient operators C types to match those of Python ints (which differ when the operands have opposite signs) and raise a ZeroDivisionError when the right operand is 0. This has up to a 35% speed penalty. If set to True, no checks are performed. Default is False.
2. @cython.boundscheck(True/False) : 
   If set to False, Cython is free to assume that indexing operations ([]-operator) in the code will not cause any IndexErrors to be raised. Lists, tuples, and strings are affected only if the index can be determined to be non-negative (or if wraparound is False). Conditions which would normally trigger an IndexError may instead cause segfaults or data corruption if this is set to False. Default is True.
3. @cython.wraparound(True/False) :
   In Python, arrays and sequences can be indexed relative to the end. For example, A[-1] indexes the last value of a list. In C, negative indexing is not supported. If set to False, Cython is allowed to neither check for nor correctly handle negative indices, possibly causing segfaults or data corruption. If bounds checks are enabled (the default, see boundschecks above), negative indexing will usually raise an IndexError for indices that Cython evaluates itself. However, these cases can be difficult to recognise in user code to distinguish them from indexing or slicing that is evaluated by the underlying Python array or sequence object and thus continues to support wrap-around indices. It is therefore safest to apply this option only to code that does not process negative indices at all. Default is True.
   

###### Modifications for the code :
1. gauss,to_dec functions are defined using cdef.
2. ckt function is defined using cpdef as we need python version of it to call it.
3. All the iterator variables are type castes as int using cdef.
4. I did free the compiler from bound checks, negative indexing checks and zero division error checks ny using compiler directives @cython.boundscheck(False), @cython.wraparound(False), @cython.cdivision(True).
5. The variables which are used as flags or counting are type casted as integers. and the variables whose type is not getting changed in the whole function are type casted respectively.
6. The lists which are used in the functions are typecasted as list. I haven't used malloc as I couldn't determine the length of the lists priorly.
7. In gauss function, norm can take complex numbers. So, it is typecasted as complex and iterators i,j,l,p,k as integers and count as integer , sol list is also defined.
8. In ckt fuunction, n amd m are not typecasted because in function their datatype is getting changed from string to integer.In ckt function, iterators i,j and start,end,freq,nv are type casted. nodes, A,b list is also typecasted as list.
9. In to_dec iterator and e_pos variables are typecasted.
10. The arguments taken by each function are also typecasted respectively

The below cell contains the function without using cython:

In [10]:
def gauss1(A,b):
    try:
        if type(A)!=list:
            A=A.tolist()
        if type(b)!=list:b=b.tolist()# making sure that the inputs are lists if not converting them to lists.
    except: return "Error! A has to be a square matrix and b has to be a column matrix and len(A) should be equal to len(b)"
    if len(A)!=len(b): return "Error! no.of elements in A has to be equal to no. of elements in b"
    arg_mat=(A.copy())
    #creatiing argumented matrix of A and b i.e., arg_mat=[A|b]
    for i in range(len(arg_mat)):
        arg_mat[i].append(b[i])  
    i=0 # step 1
    while i<len(A):
        if arg_mat[i][i]!=0 :
            n=arg_mat[i][i]
            for j in range(len(A)+1): #making the the [i][i]th element as 1 if it is non zero.
                arg_mat[i][j]/=n
        else :
            count=0
            #step 4
            for k in range(i+1,len(A)):
                if arg_mat[k][i]!=0:
                    arg_mat[k],arg_mat[i]=arg_mat[i],arg_mat[k] 
                    count+=1
                    break
            #step 5
            if count==0: 
                arg_mat[len(arg_mat)-1],arg_mat[i]=arg_mat[i],arg_mat[len(arg_mat)-1] 
                i+=1
                continue
            #step 2
            for j in range(len(A)+1):
                n=arg_mat[i][i]
                arg_mat[i][j]=(arg_mat[i][j]/n)
            #step 3 
        for p in range(len(A)):
            if p!=i:
                norm=arg_mat[p][i]
                for l in range(len(A)+1):
                    arg_mat[p][l]-=(norm)*arg_mat[i][l]
        i+=1 #step 6
   # print("The row reduced echlon form of the given system of equations is ")
   # print(arg_mat)
    sol=[0 for i in range(len(A))]
    for i in range(len(A)):
        A_part=arg_mat[i][:len(arg_mat)]
        b_part=arg_mat[i][len(arg_mat)]
        if A_part==[0 for i in range(len(A))] and b_part==0: # step 8
            return "infinitely many solutions"
        elif A_part==[0 for i in range(len(A))] and b_part!=0: #step 9
            return "no solution"
    #step 7
    for i in range(len(A)):
        for j in range(len(A)):
            if arg_mat[i][j]==1:
                sol[j]=arg_mat[i][-1]
    # print("the solution is")
    return sol
def to_dec1(s):
    e_pos=0
    for j in range(len(s)):
        if s[j]=='e':
            e_pos=j
            break
    return int(s[:j])*pow(10,int(s[j+1:]))

def ckt1(filename):
    f=open(filename,'r') #step 1: to read file
    lis=[]
    for each in f:     #step 2: to store the contents of file into list
        lis.append(each.split())
    f.close()
    start=0
    end=0
    freq=0
# to remove junk , .circuit and .end i.e., taking only components of the circuit
    for i in range(len(lis)): #step 3
        if lis[i]==['.circuit']:
            start=i
        if lis[i]==[".end"]:
            end=i
        if lis[i]!=[]: 
            if lis[i][0]==".ac": freq=2*pi*int(lis[i][2]) #converting the given frequency to angular frequency
    lis=lis[start+1:end] #step 4
#step 5 : to remove comments
    for i in range(len(lis)):
        com=0
        for j in range(len(lis[i])):
            if lis[i][j][0]=="#":
                com=j
        if com!=0:lis[i]=lis[i][:com]
    #step 6: finding all the nodes and total no. of voltage sources
    nodes=[]
    nv=0
    for i in range(len(lis)):
        if lis[i][0][0]=="V": nv+=1
        if lis[i][1]=='GND': lis[i][1]='0'
        if lis[i][2]=='GND': lis[i][2]='0'
        if lis[i][1] not in nodes:
            nodes.append(lis[i][1])
        if lis[i][2] not in nodes:
            nodes.append(lis[i][2])
    nodes.remove('0')
    #step 7 : creating coefficient and constant matrices
    A=[[complex(0) for i in range(len(nodes)+nv)] for j in range(len(nodes)+nv)]
    b=[0 for j in range(len(nodes)+nv)]
    k=0
    #step 8 : analysing elements in the matrices and making appropriate changes in the matrices
    for j in range(len(lis)):
        n=lis[j][1]
        m=lis[j][2]
        if n.isdigit() :n=int(lis[j][1])
        else : n=int(lis[j][1][1:]) #line 1
        if m.isdigit() :m=int(lis[j][2])
        else : m=int(lis[j][2][1:])
        if lis[j][0][0]=='R': # for resistor
            if n!=0 and m!=0:
                n-=1
                m-=1
                if lis[j][3].isdigit():
                    A[n][n]+=(1/int(lis[j][3]))
                    if n!=m:
                        A[n][m]-=(1/int(lis[j][3]))
                        A[m][n]-=(1/int(lis[j][3]))
                        A[m][m]+=(1/int(lis[j][3]))
                else:
                    A[n][n]+=(1/to_dec1(lis[j][3]))
                    if n!=m:
                        A[n][m]-=(1/to_dec1(lis[j][3]))
                        A[m][n]-=(1/to_dec1(lis[j][3]))
                        A[m][m]+=(1/to_dec1(lis[j][3]))
            elif n==0:
                if lis[j][3].isdigit(): A[m-1][m-1]+=(1/int(lis[j][3]))
                else : A[m-1][m-1]+=(1/to_dec1(lis[j][3]))
            elif m==0:
                if lis[j][3].isdigit(): A[n-1][n-1]+=(1/int(lis[j][3]))
                else : A[n-1][n-1]+=(1/to_dec1(lis[j][3]))
        elif lis[j][0][0]=='L': # for inductors
            if n!=0 and m!=0:
                n-=1
                m-=1
                if lis[j][3].isdigit():
                    A[n][n]+=(1/(int(lis[j][3])*freq*(1j)))
                    if n!=m:
                        A[n][m]-=(1/(int(lis[j][3])*freq*(1j)))
                        A[m][n]-=(1/(int(lis[j][3])*freq*(1j)))
                        A[m][m]+=(1/(int(lis[j][3])*freq*(1j)))
                else:
                    A[n][n]+=(1/to_dec1(lis[j][3]))
                    if n!=m:
                        A[n][m]-=(1/(to_dec1(lis[j][3])*freq*(1j)))
                        A[m][n]-=(1/(to_dec1(lis[j][3])*freq*(1j)))
                        A[m][m]+=(1/(to_dec1(lis[j][3])*freq*(1j)))
            elif n==0:
                if lis[j][3].isdigit(): A[m-1][m-1]+=(1/(int(lis[j][3])*freq*(1j)))
                else : A[m-1][m-1]+=(1/(to_dec1(lis[j][3])*freq*(1j)))
            elif m==0:
                if lis[j][3].isdigit(): A[n-1][n-1]+=(1/(int(lis[j][3])*freq*(1j)))
                else : A[n-1][n-1]+=(1/(to_dec1(lis[j][3])*freq*(1j)))
        elif lis[j][0][0]=='C': # for capacitors
            if n!=0 and m!=0:
                n-=1
                m-=1
                if lis[j][3].isdigit():
                    A[n][n]+=((int(lis[j][3])*freq*(1j)))
                    if n!=m:
                        A[n][m]-=((int(lis[j][3])*freq*(1j)))
                        A[m][n]-=((int(lis[j][3])*freq*(1j)))
                        A[m][m]+=((int(lis[j][3])*freq*(1j)))
                else:
                    A[n][n]+=(to_dec1(lis[j][3]))
                    if n!=m:
                        A[n][m]-=(to_dec1(lis[j][3])*freq*(1j))
                        A[m][n]-=(to_dec1(lis[j][3])*freq*(1j))
                        A[m][m]+=(to_dec1(lis[j][3])*freq*(1j))
            elif n==0:
                if lis[j][3].isdigit(): A[m-1][m-1]+=cmath.rect(freq*int(lis[j][3]),pi/2)
                else : A[m-1][m-1]+=((to_dec1(lis[j][3])*freq*(1j)))
            elif m==0:
                if lis[j][3].isdigit(): A[n-1][n-1]+=((int(lis[j][3])*freq*(1j)))
                else : A[n-1][n-1]+=((to_dec1(lis[j][3])*freq*(1j)))
        elif lis[j][0][0]=='V': #for voltage sources n1(n) is positive n2(m) is negative
            if m!=0:           #current through voltage source is directed from n1(n) to n2(m)
                m-=1
                A[len(nodes)+k][m]-=1
                A[m][len(nodes)+k]-=1
            if n!=0:
                n-=1
                A[len(nodes)+k][n]+=1
                A[n][len(nodes)+k]+=1
            if lis[j][3]=="dc" : b[len(nodes)+k]+=int(lis[j][4])
            elif lis[j][3]=="ac" :b[len(nodes)+k]+=cmath.rect(int(lis[j][4]),int(lis[j][5]))
            k+=1
        elif lis[j][0][0]=='I': # for current sources flowing from n2 to n1 i.e.s flowing into n1 and out of n2
            if m!=0:
                m-=1
                if lis[j][3]=="dc" :b[m]-=int(lis[j][4])
                elif lis[j][3]=="ac" : b[m]-=cmath.rect(int(lis[j][4]),int(lis[j][5]))
            if n!=0:
                n-=1
                if lis[j][3]=="dc" :b[n]+=int(lis[j][4])
                elif lis[j][3]=="ac" :b[n]+=cmath.rect(int(lis[j][4]),int(lis[j][5]))
    return gauss1(A,b)      #step 9 : To find the soltion of the matrices   

In [11]:
print(ckt('ckt1.netlist'))
print(ckt1('ckt1.netlist'))

[0j, 0j, 0j, (-5+0j), (-0.0005-0j)]
[0j, 0j, 0j, (-5+0j), (-0.0005-0j)]


In [13]:
%timeit ckt('ckt1.netlist') # using cython
%timeit ckt1('ckt1.netlist') # pure python

91.9 µs ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
196 µs ± 6.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


- The optimised version (with cython) is nearly twice faster than the original one.
- Don't write any comment beside %load_ext Cython, it throws No module found error

###### NOTE:
- First import the libraries which u require.
- Run the user defined function before you use it in some other cell.

- This is an .ipynb file, which we can run in Jupyter notebook or Jupyter lab.
- In Jupyter lab, we need to upload this document to the workspace and start editting and running.
- In jupyter notebook, which is a local host of our pc, we need to know where the file is located, opening this file is same as we do in file manager.
- To open in jupyter notebook, first we need to unzip the file i.e., extract all the files from it and open the ipynb file.